In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
# from PIL import ImageGrab

In [2]:
# imgElon = face_recognition.load_image_file('Images/Elon_Musk.jpg')
# imgElon = cv2.cvtColor(imgElon,cv2.COLOR_BGR2RGB) #converting it to RGB
# imgTest = face_recognition.load_image_file('Images/Elon_test.jpeg')
# imgTest = cv2.cvtColor(imgTest,cv2.COLOR_BGR2RGB) 

# faceLoc = face_recognition.face_locations(imgElon)[0] 
# encodeElon = face_recognition.face_encodings(imgElon)[0]

In [3]:
path = 'ImagesAttendance'
images = []
classNames = []
myList = os.listdir(path)
# print(myList)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}') #cl is the name of our image
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames) #name of images without encodings

['Bill_Gates', 'deekshant_rawat', 'Elon_Musk', 'Jack_ma', 'Jeff_Bezos']


In [4]:
#encoding process : we  have to find encoding for each one of them.

#we can create a simple function for this

def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

encodeListKnown = findEncodings(images)
print('Encoding Complete')
 


Encoding Complete


In [5]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f: 
        #storing the vlaues in cvs file, we will store Name and Time
        myDataList = f.readlines()
        #reading all the lines that we currently have in our data or file bcz if someone is already arrived we do not want to repeat it
#         print(myDataList)
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dateString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dateString}')

In [ ]:
#Step 3: Find the matches between our encodings 
#but we dont have an image to macth it with, now that image will be coming 
#from our webcam

#initialising the web cam

cap = cv2.VideoCapture(0)

#to get each frame one by one 
while True:
    success, img = cap.read()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25) #to speed up the process reducing the size of image
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    

    
#in the webcam we may find multiple face so for that we are going to find the location of our faces then we are going to send in these locations 
#to our encoding function to find locations:
    facesCurFrame = face_recognition.face_locations(imgS)
    
    # next step is to find the encoding of our web cam image
    encodesCurFrame = face_recognition.face_encodings(imgS,facesCurFrame)
    
#now moving to our main 3rd step where we are finding the matches

#iterate through all the faces that we have found in our current frame and then compare all these faces with all
#the encodings that we found before

    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame): #one by one it will grab one face location from the facesCurFrame list and the it will grab the encoding of encodeFace from encodesCurFrame
    #we want them in the same loop that's why we are using them
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #we are sending a list encodeListKnown in our face_distance func so it will return us a list as well
        #it will give us 4 values as we have 4 values as we have 4 known faces and it will give us the distance of 
        #each one of them,so the lowest distance will be our best match
        
        #print(faceDis)
        matchIndex = np.argmin(faceDis) #index at which our face distance is min
        
        #once we have that index we know which person we are talking about
        
        #so we can display a bounding box around them and we can write their name
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            #print(name)
            #surrounding face with box
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4 #as we have scaled down the images so again increasing its size
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendance(name)
    cv2.imshow('Webcam',img)
    cv2.waitKey(1)
        